# Final Analysis for Publication

This is the final analysis.

## Data Import

see: `etl/1_download_all_years_cms.py` 


In [1]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:default:

## Number of 4 and 5 star facilties with IJ on their record

Script Language:

> OUR SCRIPPS NEWS ANALYSIS OF GOVERNMENT DATA FROM FEBRUARY 2024 FOUND 247 NURSING HOMES ACROSS THE UNITED STATES...IN SIMILAR SITUATIONS -- HOLDING TOP OVERALL RATINGS -- A FOUR OR A FIVE - DESPITE "IMMEDIATE JEOPARDY” FINDINGS THAT RESIDENTS HAD BEEN PLACED AT RISK IN THE LAST THREE YEARS -- SINCE FEBRUARY 2021.

Data Analysis in duckdb. Run inside the `data/source/nursinghome-compare/2024/02` subdirectory.

In [2]:
%%sql

SELECT min(cast(replace("1 star", '>', '') as double)) FROM '../data/source/nursinghome-compare/2024/02/NH_HlthInspecCutpointsState_Feb2024.parquet';

,"min(CAST(replace(""1 star"", '>', '') AS DOUBLE))"
0,33.667


In [3]:
%%sql

/* This filters "NH_HealthCitations_Feb2024.parquet" for immediate jeopardy findings.
Scope severity codes J and above constitute findings of "immediate jeopardy" (IJ). 
J is for isolated cases of immediate jeopardy, K is for a pattern, and L is for if those cases are widespread.*/

CREATE table facilities_wih_ij as   
SELECT distinct ccn 
from ( 
    SELECT DISTINCT "CMS CERTIFICATION NUMBER (CCN)" as ccn 
    FROM '../data/source/nursinghome-compare/2024/02/NH_HealthCitations_Feb2024.parquet'
    WHERE ( "Scope Severity Code" LIKE 'J' OR  "Scope Severity Code" LIKE 'K' OR  "Scope Severity Code" like 'L')
	AND CAST("Survey Date" as date) >= CAST( '2021-02-01' as date));

SELECT COUNT(*) from facilities_wih_ij;

,count_star()
0,3597


In [4]:
%%sql

/* This counts the number of facilities in the February 2024 snapshot with an overall rating of 4 or higher. */

SELECT COUNT(*) 
FROM "../data/source/nursinghome-compare/2024/02/NH_ProviderInfo_Feb2024.parquet" 
	INNER JOIN facilities_wih_ij ON "CMS CERTIFICATION NUMBER (CCN)" = ccn
WHERE CAST("Overall Rating" AS INTEGER) >= 4 ;

,count_star()
0,247


In [5]:
%%sql

/* This counts the number of facilities in the February 2024 snapshot with a health inspection rating of 4 or higher. */

SELECT COUNT(*)

FROM "../data/source/nursinghome-compare/2024/02/NH_ProviderInfo_Feb2024.parquet" INNER JOIN
 	facilities_wih_ij ON "CMS CERTIFICATION NUMBER (CCN)" = ccn
WHERE CAST("Health Inspection Rating" AS INTEGER) >= 4;

,count_star()
0,153


In [6]:
%%sql

/* This counts the number of facilities in the February 2024 snapshot with a health inspection rating of 5. */

SELECT COUNT(*)
from "../data/source/nursinghome-compare/2024/02/NH_ProviderInfo_Feb2024.parquet" INNER JOIN
 	facilities_wih_ij on "CMS CERTIFICATION NUMBER (CCN)" = ccn
WHERE CAST("Health Inspection Rating" AS INTEGER) = 5;

,count_star()
0,7


In [7]:
%%sql
/* This table counts the number of facilities (count_star()) within each rating tier (overall_rating) and the average number of penalties (mean(CAST("Total Number of Penalties" AS INTEGER))) within that tier. */

SELECT "Overall Rating", count(*), mean(CAST("Total Number of Penalties" as INTEGER)) 
from "../data/source/nursinghome-compare/2024/02/NH_ProviderInfo_Feb2024.parquet" INNER JOIN 
	facilities_wih_ij ON "CMS CERTIFICATION NUMBER (CCN)" = ccn
GROUP BY 1 
ORDER BY 1;


,Overall Rating,count_star(),"mean(CAST(""Total Number of Penalties"" AS INTEGER))"
0,1,1883,5.154541
1,2,1001,3.543457
2,3,393,2.918575
3,4,180,2.511111
4,5,67,1.716418
5,None,73,8.383562


## Rating histories

Script language:
>OUR INVESTIGATION FOUND THE FACILITY STILL HELD ON TO ITS TOP OVERALL RATING...5 STARS...FOR SEVEN MONTHS AFTER KAREN DIED

Karen died December 13, 2022. Touchmark on South Hill's rating dropped to a 4 July 1st, 2022.


In [8]:
# Read our etl file
fd = open('../etl/2_ratings_sql_macro.sql', 'r')
sqlFile = fd.read()
fd.close()

# It's a single command
sqlCommand = sqlFile.split(';')[0]

try:
        duckdb.sql(sqlCommand)
except:
        print("Command skipped")

In [9]:
/* Touchmark ratings */
%%sql

select * from ratings_by_provider_over_time('505498');

SyntaxError: invalid syntax (3316142130.py, line 1)

Two other homes -- "Willowbrooke Court" of Pennsylvania and "Good Samaritan Society" of Minnesota are mentioned in the piece as maintaining a 4 or 5 star overall rating despite immediate jeopardy findings.

Script language:

>IN PENNSYLVANIA...RECORDS AND PHOTOS SCRIPPS NEWS OBTAINED REVEAL AN ELDERLY WOMAN FROZE TO DEATH OUTSIDE HER NURSING HOME – IN 2021- AFTER STAFF FAILED TO NOTICE SHE WAS MISSING.

>AND IN MINNESOTA THAT SAME YEAR...A RESIDENT SLIPPED OUT OF his FACILITY, BOUGHT A KNIFE, AND KILLED HIMSELF. AN INVESTIGATION FOUND STAFF FAILED TO CALL POLICE FOR MORE THAN 30 MINUTES TO REPORT THAT THE resident HAD LEFT THE SITE.

>SINCE THE INCIDENTS....BOTH FACILITIES HAVE MAINTAINED A TOP RATING – EITHER A FOUR OR A FIVE.

In [ ]:
%sql select * from ratings_by_provider_over_time('395490'); 

,Federal Provider Number,Provider Name,Provider State,Overall Rating,Health Inspection Rating,Processing Date
0,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2020-08-01
1,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2020-09-01
2,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2020-10-01
3,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2020-11-01
4,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2020-11-01
5,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2021-01-01
6,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2021-02-01
7,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2021-03-01
8,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2021-04-01
9,395490,WILLOWBROOKE COURT-SOUTHAMPTON,PA,5,4,2021-05-01


In [ ]:
%sql select * from ratings_by_provider_over_time('245441');

,Federal Provider Number,Provider Name,Provider State,Overall Rating,Health Inspection Rating,Processing Date
0,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2020-08-01
1,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2020-09-01
2,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2020-10-01
3,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2020-11-01
4,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2020-11-01
5,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2021-01-01
6,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2021-02-01
7,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2021-03-01
8,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2021-04-01
9,245441,GOOD SAMARITAN SOCIETY - ALBERT LEA,MN,5,4,2021-05-01


##  5-star-to-1-star in other states

Script language:
> A SCRIPPS NEWS ANALYSIS FOUND 397 FOUR AND FIVE STAR NURSING HOMES ACROSS THE COUNTRY COULD POTENTIALLY HAVE JUST ONE STAR BY ANOTHER STATE’S STANDARDS.

In [10]:
%%sql

create table ratings_and_scores as
select "CMS CERTIFICATION NUMBER (CCN)" ccn, 
	cast("Overall Rating" as integer) overall, 
	cast( "Health Inspection Rating" as integer) inspection,
	cast( "QM Rating" as integer) qm,
	cast( "Long-Stay QM Rating"  as integer) long_stay_qm,
	cast( "Staffing Rating" as integer) staffing, 
	cast( "Total Weighted Health Survey Score" as double ) score, 
	"Abuse Icon" as abuse,
	"Provider Name" provider_name, 
	"State" provider_state
from "../data/source/nursinghome-compare/2024/02/NH_ProviderInfo_Feb2024.parquet";

/* A table of the minimum cutoffs across all states for each star tier */
create table cutoffs as
select
	min(cast(replace("1 star", '>', '') as double)) one_star,
	min(cast(left("2 stars", instr("2 stars", '-') - 1) as double ) )  two_star,
	min(cast(left("3 stars", instr("3 stars", '-') - 1) as double ) ) three_star,
	min(cast(left("4 stars", instr("4 stars", '-') - 1) as double ) ) four_star
from '../data/source/nursinghome-compare/2024/02/NH_HlthInspecCutpointsState_Feb2024.parquet';

/* The overall rating is calculated by combining health inspection rating with the staffing and QM ratings. 
This table recalculates that score using the most stringent state's 1-star cutoff. HIGHER RATINGS ARE WORSE.*/

create table worst_ratings as
Select *, worst_inspection + staffing_modifier + qm_modifier as worst_rating
From (Select ratings_and_scores.*,
       case when score >= (select one_star   from cutoffs) then 1 
	    when score >= (select two_star   from cutoffs) then 2
	    when score >= (select three_star from cutoffs) then 3
	    when score >= (select four_star  from cutoffs) then 4
	    else 5 end as worst_inspection,
	case staffing when 5 then 1 when 1 then -1 else 0 end as staffing_modifier,
	case qm when 5 then 1 when 1 then -1 else 0 end as qm_modifier
       From ratings_and_scores);

,Success


In [12]:
%sql select * from cutoffs

,one_star,two_star,three_star,four_star
0,33.667,20.668,13.334,3.334


In [13]:
%sql select overall, worst_inspection, count(*) from worst_ratings group by 1, 2 order by 2, 1;

,overall,worst_inspection,count_star()
0,1.0,1,3388
1,2.0,1,2700
2,3.0,1,1884
3,4.0,1,1036
4,5.0,1,513
5,NaN,1,87
6,1.0,2,73
7,2.0,2,283
8,3.0,2,620
9,4.0,2,658


In [14]:
# Overall ratings (rows) compared to calculated ratings with the lowest cutpoint for health inspections (columns)
%sql PIVOT worst_ratings ON worst_rating USING COUNT(*) GROUP BY OVERALL;

,overall,-1,0,1,2,3,4,5,6,7
0,NaN,0,0,87,0,0,0,59,0,0
1,1.0,328,1949,1190,1,1,0,0,0,0
2,2.0,14,492,1951,587,20,2,0,0,0
3,3.0,1,117,1287,1179,286,2,1,0,0
4,4.0,0,3,386,1079,669,444,7,0,0
5,5.0,0,0,8,426,559,756,675,265,62


In [15]:
# The number of homes national currently have 4 or 5 overall star ratings but would have a 1 star rating in the state with the lowest cut-point (Alabama)
%sql select count(*) from worst_ratings where overall >= 4 and worst_rating <= 1;

,count_star()
0,397


In [19]:
# The above counts grouped by state
%sql select provider_state, count(*) from worst_ratings where overall >= 4 and worst_rating <= 1 group by 1 order by 2 desc;

,provider_state,count_star()
0,CA,78
1,IL,51
2,MI,38
3,MO,30
4,WA,26
5,AR,25
6,OH,23
7,MA,20
8,MD,17
9,WV,14


In [17]:
%%sql
/* State cutoffs */
select "State",
	min(cast(replace("1 star", '>', '') as double)) one_star,
	min(cast(left("2 stars", instr("2 stars", '-') - 1  ) as double ) )  two_star,
	min(cast(left("3 stars", instr("3 stars", '-') - 1  ) as double ) ) three_star,
	min(cast(left("4 stars", instr("4 stars", '-') - 1  ) as double ) ) four_star
from '../data/source/nursinghome-compare/2024/02/NH_HlthInspecCutpointsState_Feb2024.parquet'
GROUP by 1
Order by 2 desc;

,State,one_star,two_star,three_star,four_star
0,DC,224.500,138.001,70.668,31.334
1,NM,218.000,140.668,93.334,44.001
2,IL,187.750,101.668,60.001,25.334
3,WA,183.083,118.834,78.001,42.668
4,MI,165.167,92.334,57.334,26.668
5,RI,153.333,81.001,46.668,18.668
6,CA,149.333,96.668,64.668,34.668
7,MO,144.500,75.334,47.334,24.001
8,VT,139.333,50.668,16.668,3.334
9,HI,138.667,92.668,58.334,38.668
